<a href="https://colab.research.google.com/github/ShiyuDeng-Simon/MeddiConnect/blob/main/LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>